In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

Matplotlib created a temporary cache directory at /tmp/matplotlib-b88r623n because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Let's try the idea of GPTPU

## Creating a model from a "concrete function"

In [2]:
do_render_code("gptpu_create.py", show = ["BMM","return"])

// gptpu_create.py:5-15 (11 lines)
class BMM(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=(24), dtype=tf.float32)])
    def __call__(self, input):
        a = input[0:16]
        b = input[16:24]
        a = tf.reshape(a, [4,4])
        b = tf.reshape(b, [4,2])
        c = tf.linalg.matmul(a,b)
        divisor = 2.0
        c = tf.math.divide_no_nan(c,divisor)
        return c

In [3]:
do_render_code("gptpu_create.py", show="main")

// gptpu_create.py:19-48 (30 lines)
def main():
    np.random.seed(10)
    for x in range(5):
        a = np.random.randint(2,size=16)
        b = np.random.randint(2,size=8)
        input = np.append(a,b)
        a = np.reshape(a, (4,4))
        b = np.reshape(b, (4,2))
        start = time.perf_counter()
        c = np.matmul(a,b)
        c = c >> 1
        inference_time = time.perf_counter() - start
        print('%.2f us' % (inference_time*1000000))
        # Debugging
        result = 0
        c = np.reshape(c, (8))
        for y in range(8):
            result = (result << 1) + c[y]
        print(result)
    input = np.random.randint(2,size=24)
    model = BMM()
    concrete_func = model.__call__.get_concrete_function()
    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func],model)
    tflite_model = converter.convert()

    # Save the model.
    with open('bmm_gptpu.tflite', 'wb') as f:
      f.write(tflite_model)
if __name__ == '__main__':
  main()

In [4]:
! cs277 edgetpu run "python3 gptpu_create.py"

htseng
ssh htseng@horsea "cd /datahub/htseng/2024wi/demo/GPTPU; srun -N1 -p edgetpu python3 gptpu_create.py"
2024-01-23 15:18:47.346231: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 15:18:47.853291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-23 15:18:48.513391: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-23 15:18:48.612035: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-01-23 15:18:48.612067: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-01-23 15:18:48.61

## Inference the created model

In [5]:
do_render_code("bmm_gptpu.py")

// bmm_gptpu.py:1-41 (41 lines)
import numpy as np
import tensorflow as tf
import time
from pycoral.adapters import classify
from pycoral.adapters import common
from pycoral.utils.dataset import read_label_file
from pycoral.utils.edgetpu import make_interpreter 

def main():
#    interpreter = tf.lite.Interpreter(model_path="bmm_npu.tflite")
    interpreter = make_interpreter("bmm_gptpu.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
# Run inference
    print('----INFERENCE TIME----')
    print('Note: The first inference on Edge TPU is slow because it includes',
            'loading the model into Edge TPU memory.')
    input_data = []
    output_data = []
    np.random.seed(10)
    for x in range(5):
        a = np.random.randint(2,size=16)
        b = np.random.randint(2,size=8)
        input_data = np.append(input_data,np.append(a,b))
    input_data = np.float32(np.reshape(input_data,(5,24)))
    
    for x in range(5):
        start = time.perf_counter()
#        print(input_data[x])
        interpreter.set_tensor(input_details[0]['index'], np.reshape(input_data[x],(24)))
        interpreter.invoke()
        output_data = np.append(output_data, interpreter.get_tensor(output_details[0]['index']))
#        print(output_data[x].astype(np.int32))
        inference_time = time.perf_counter() - start
        print('%.2f us' % (inference_time*1000000))
#        print(result)
    print(tf.reshape(output_data.astype(np.uint8),[5,8]))
    
if __name__ == '__main__':
  main()

In [8]:
! cs277 edgetpu run "python3 bmm_gptpu.py"

htseng
ssh htseng@horsea "cd /datahub/htseng/2024wi/demo/GPTPU; srun -N1 -p edgetpu python3 bmm_gptpu.py"
2024-01-23 15:19:33.582395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 15:19:34.089285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-23 15:19:34.738068: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
----INFERENCE TIME----
Note: The first inference on Edge TPU is slow because it includes loading the model into Edge TPU memory.
29.05 us
9.37 us
6.73 us
6.21 us
8.35 us
tf.Tensor(
[[0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0]], shape

## Let's compare with NPU on Edge TPU

In [9]:
! cs277 edgetpu run "python3 bmm_npu_edgetpu.py"

htseng
ssh htseng@horsea "cd /datahub/htseng/2024wi/demo/GPTPU; srun -N1 -p edgetpu python3 bmm_npu_edgetpu.py"
2024-01-23 15:20:44.632407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 15:20:45.145164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
----INFERENCE TIME----
Note: The first inference on Edge TPU is slow because it includes loading the model into Edge TPU memory.
37.27 us
11.79 us
8.51 us
7.98 us
7.32 us
[  0 117   4   0  81]
